## Code fourni pour lire les binaires:

In [281]:
import matplotlib.pyplot as plt
import numpy as np
import time

def read_int(f):
    ba = bytearray(4)
    f.readinto(ba)
    prm = np.frombuffer(ba, dtype=np.int32)
    return prm[0]

def read_double(f):
    ba = bytearray(8)
    f.readinto(ba)
    prm = np.frombuffer(ba, dtype=np.double)
    return prm[0]

def read_double_tab(f, n):
    ba = bytearray(8*n)
    nr = f.readinto(ba)
    if nr != len(ba):
        return []
    else:
        prm = np.frombuffer(ba, dtype=np.double)
        return prm

def get_pics_from_file(filename):
    # Lecture du fichier d'infos + pics detectes (post-processing KeyFinder)
    #print("Ouverture du fichier de pics "+filename)
    f_pic = open(filename, "rb")
    info = dict()
    info["nb_pics"] = read_int(f_pic)
    #print("Nb pics par trame: " + str(info["nb_pics"]))
    info["freq_sampling_khz"] = read_double(f_pic)
    #print("Frequence d'echantillonnage: " + str(info["freq_sampling_khz"]) + " kHz")
    info["freq_trame_hz"] = read_double(f_pic)
    #print("Frequence trame: " + str(info["freq_trame_hz"]) + " Hz")
    info["freq_pic_khz"] = read_double(f_pic)
    #print("Frequence pic: " + str(info["freq_pic_khz"]) + " kHz")
    info["norm_fact"] = read_double(f_pic)
    #print("Facteur de normalisation: " + str(info["norm_fact"]))
    tab_pics = []
    pics = read_double_tab(f_pic, info["nb_pics"])
    nb_trames = 1
    while len(pics) > 0:
        nb_trames = nb_trames+1
        tab_pics.append(pics)
        pics = read_double_tab(f_pic, info["nb_pics"])
    #print("Nb trames: " + str(nb_trames))
    f_pic.close()
    return tab_pics, info

    """
    ######### Pics ############
    # NO KEY
    plt.figure(1)
    plt.subplot(211)
    plt.plot(range(1,info["nb_pics"]+1), np.mean(pics_nokey,axis=0), 'ko')
    plt.xlabel('numéro de pic')
    plt.ylabel('valeur du pic')
    plt.title('no key')
    plt.ylim(0, 1.5)
    plt.grid(b=True, which='both')
    # PAD-0
    plt.subplot(212)
    plt.plot(range(1,info["nb_pics"]+1), pics_nokey[0], 'ko')
    plt.xlabel('numéro de pic')
    plt.ylabel('valeur du pic')
    plt.title('PAD-0')
    plt.ylim(0, 1.5)
    plt.grid(b=True, which='both')
    #
    plt.show()
    """

# Notre Algorithme
* Séparer les trames en chunk ------- While == précédent + [un certain pas]
* Attention à shift+a par exemple --- Maybe in top best letters ?
* ctrl alt suppr au début ---- Histoire de pas oublier

### Bin to png

In [282]:
#for file in glob.glob("../data/pics_*.bin"):
 #   key = file.split("pics_")[1].replace(".bin","")
 #   values, info = get_pics_from_file(file)
 #   plt.imshow(values,aspect="auto",)
 #   plt.savefig("../data/PNGs/pics_" + key + ".png")

### Bibliothèques utilisées :

In [283]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score
import glob

## Constitution du dataset

In [284]:
# Il faut enlever le bruit

X,Y = [],[]

pics_nokey, info = get_pics_from_file("../data/pics_NOKEY.bin")

letters = 0
nokey = np.mean(pics_nokey,axis=0)
for file in glob.glob("../data/pics_*.bin"):
    key = file.split("pics_")[1].replace(".bin","")
    if key == "LOGINMDP":
        continue
    values, info = get_pics_from_file(file)
    np.random.shuffle(values)
    values = values[:3000]
    values = np.subtract(values,nokey)
    mean = np.mean(values,axis=0)
    values = values * mean
    for frame in values:
        X.append(frame)
        Y.append(key)
    letters += 1
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)

# Scaling des données
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

### KNN Algorithm

In [285]:
import math
# Best value for N (internet said this)
N = math.floor(math.sqrt(len(Y_test)))
N = math.floor(math.sqrt(47))
if N % 2 == 0:
    N -= 1
print(N)

5


In [286]:
# KNN Clustering
classifier = KNeighborsClassifier(n_neighbors=40, p=2, metric='euclidean',n_jobs=-1)
classifier.fit(X_train,Y_train)
Y_pred = classifier.predict(X_test)
print(Y_pred)

#### K-cross validation ###
#from sklearn.model_selection import cross_val_score
#cv_scores = cross_val_score(classifier,X,Y, cv=10)
#print(cv_scores)
#print(np.mean(cv_scores))

### Evaluating model ###
print(Y_test)
print(accuracy_score(Y_test,Y_pred),"%")
#cm = confusion_matrix(Y_test,Y_pred)
# La diagonale c'est les bons guess et le reste les fautes
#print(cm)

['Y' '5' 'P' ... 'S' 'C' 'L']
['T', '5', 'P', 'X', 'Y', 'L', 'M', 'Q', '8', 'CTRL', 'U', 'SUPPR', 'SPACE', 'N', '6', 'N', 'SHIFT', '2', '6', 'ENTER', 'Z', 'W', 'Y', 'P', 'Z', 'F', '7', 'B', 'C', '8', 'H', 'A', 'N', 'NOKEY', 'D', 'E', 'ENTER', 'E', 'I', 'M', '6', 'L', 'V', 'Y', '5', 'ENTER', '2', '8', 'A', 'P', 'H', '9', 'K', 'I', 'V', 'B', 'P', 'SPACE', 'U', 'E', 'V', '1', 'F', '7', 'B', 'H', 'I', 'U', '2', '6', 'NOKEY', 'D', '5', '8', 'Y', 'W', 'J', 'M', 'Q', '1', '3', '3', 'SPACE', 'E', '5', 'Q', 'W', 'S', 'Q', 'E', '3', '1', '3', 'Y', 'SUPPR', 'N', 'V', 'N', 'F', 'L', 'D', 'W', 'M', 'C', '2', 'I', '0', 'U', 'Y', 'C', 'E', 'NOKEY', '7', 'ENTER', '6', 'T', 'U', 'H', 'M', '9', 'B', 'V', 'I', 'Q', '7', 'SPACE', 'CTRL', 'L', '6', 'Y', '5', 'S', 'N', 'J', 'G', 'G', 'L', '3', 'M', 'P', 'X', '7', 'Y', 'CTRL', 'I', 'M', 'H', 'W', 'S', '6', 'I', 'CTRL', 'K', 'Z', 'V', 'Z', 'J', 'M', 'Y', 'W', 'G', 'CTRL', 'SUPPR', 'R', 'P', 'G', '3', '0', 'O', '8', 'Q', 'Z', 'B', '3', 'L', 'A', '7', 'P', '1',

In [221]:
""" Code poubelle -> tests
pics_nokey, info = get_pics_from_file("../data/pics_NOKEY.bin")
values, info = get_pics_from_file("../data/pics_3.bin")
print(np.mean(values,axis=0))
np.random.shuffle(values)
plt.imshow(values[:3000],aspect="auto")
plt.show()
nokey = np.mean(pics_nokey,axis=0)
denoised = np.subtract(values[:3000],nokey)
plt.imshow(denoised,aspect="auto")
plt.show()
mean = np.mean(values[:3000],axis=0)
cleaned = values[:3000] * mean
plt.imshow(cleaned,aspect="auto")
plt.show()

denoised = np.subtract(cleaned,nokey)
plt.imshow(denoised,aspect="auto")
plt.show()
"""

' Code poubelle -> tests\npics_nokey, info = get_pics_from_file("../data/pics_NOKEY.bin")\nvalues, info = get_pics_from_file("../data/pics_3.bin")\nprint(np.mean(values,axis=0))\nnp.random.shuffle(values)\nplt.imshow(values[:3000],aspect="auto")\nplt.show()\nnokey = np.mean(pics_nokey,axis=0)\ndenoised = np.subtract(values[:3000],nokey)\nplt.imshow(denoised,aspect="auto")\nplt.show()\nmean = np.mean(values[:3000],axis=0)\ncleaned = values[:3000] * mean\nplt.imshow(cleaned,aspect="auto")\nplt.show()\n\ndenoised = np.subtract(cleaned,nokey)\nplt.imshow(denoised,aspect="auto")\nplt.show()\n'

In [339]:
guess, info = get_pics_from_file("../data/pics_LOGINMDP.bin")
output = []
pas = 95

for i in range(10000):
    if (pas*(i+1)) >= len(guess):
        break
    curr = guess[pas*i:pas*(i+1)]
    curr = np.subtract(curr,nokey)
    mean = np.mean(curr,axis=0)
    curr = curr * mean
    curr = sc_X.transform(curr)
    curr_output = classifier.predict(curr)
    unique,pos = np.unique(curr_output,return_inverse=True)
    max = unique[np.bincount(pos).argmax()]
    if max == 'NOKEY':
        continue
    if max == 'SHIFT':
        curr = guess[pas*i:pas*(i+1)]
        #erase pic 5 (shift)
        for j,frame in enumerate(curr):
            frame[5] = 0
            curr[j] = frame
        curr = np.subtract(curr,nokey)
        mean = np.mean(curr,axis=0)
        curr = curr * mean
        curr = sc_X.transform(curr)
        curr_output = classifier.predict(curr)
        unique,pos = np.unique(curr_output,return_inverse=True)
        max = unique[np.bincount(pos).argmax()]
        if max == 'NOKEY' or max == 'SHIFT':
            continue
        if output == [] or ("SHIFT " + max) != output[-1]:
            output.append("SHIFT " + max)
    elif output == [] or max != output[-1]:
        output.append(max)

#probs = classifier.predict_proba(input)
#best_n = np.argsort(-probs, axis=1)[:, :3]
#print(best_n)

CTRL 7
7 CTRL
S 7
X SHIFT S
E SHIFT X
T SHIFT E
3 SHIFT T
A SHIFT 3
M SHIFT A
I SHIFT M
H SHIFT I
U H
A U
E A
D E
K D
Q K
T Q
O T
N O
2 N
0 2
2 0
3 2
ENTER 3


In [338]:
print("len: ",len(output))
for a in output:
    print(a)


len:  24
7
CTRL
SHIFT S
SHIFT E
SHIFT S
SHIFT Y
SHIFT S
SHIFT A
SHIFT M
SHIFT S
SHIFT I
SHIFT 3
B
A
D
K
A
Y
O
N
3
0
2
ENTER
